In [1]:
from collections import namedtuple, deque
import random
from sklearn.preprocessing import LabelEncoder
import torch
import torchrl
import torch.optim as optim
import torch.nn.functional as F
import numpy as np


In [7]:
class Cell(object):
    def __init__(self, pos, board_dims):
        self.self_pos = pos
        self.value = '-'
        self.token = ''
        self.board_dims = board_dims #[x,x]
        self.normal_neighbors = {} #key direction, val cell
        self.diag_neighbors = {} #key direction, val cell
        self.directions = ['top','bottom','right','left','top right','bottom left','bottom right','top left']
        
    def get_neighbors(self):
        return [self.normal_neighbors, self.diag_neighbors]
    
    def get_neighbor_poses(self):
        normal_poses = []
        diag_poses = []
        
        for normal in self.normal_neighbors:
            normal_poses.append(self.normal_neighbors[normal].self_pos)
        for diag in self.diag_neighbors:
            diag_poses.append(self.diag_neighbors[diag].self_pos)
        return normal_poses, diag_poses
    
    def add_diag_neighbor(self, neighbors):
        for neighbor in neighbors:
            self.diag_neighbors.update({neighbor:neighbors[neighbor]})
    
    def add_normal_neighbor(self, neighbors):
        for neighbor in neighbors:
            self.normal_neighbors.update({neighbor:neighbors[neighbor]})
    
    def get_pos(self):
        return self.self_pos
    
    def flip(self, value, token):
        self.value = value
        self.token = token
        
    def get_neighbor_cell_in_direction(self, direction):
        if direction in list(self.normal_neighbors.keys()):
            return self.normal_neighbors[direction]
        elif direction in list(self.diag_neighbors.keys()):
            return self.diag_neighbors[direction]
        else:
            return False
        
    def get_empty_neighbors(self):
        #list of cells
        empty_neighbors = []
        for direction in self.directions:
            neighbor_in_direction = self.get_neighbor_cell_in_direction(direction)
            if neighbor_in_direction == False:
                continue
            else:
                if neighbor_in_direction.value == '-':
                    empty_neighbors.append(neighbor_in_direction)
        return empty_neighbors

In [4]:

class Board(object):
    def __init__(self, board_dims=[8,8]):
        self.board_dims = board_dims
        self.board = [[Cell(pos=[row,col],board_dims=self.board_dims) for col in range(board_dims[0])] for row in range(board_dims[1])]
        self.fill_neighbors()
        
    def fill_neighbors(self):
        #build board with all dependencies, like nieghbors and locations
        for row in self.board:
            for cell in row:
                diags, normals = self.get_neighbor_poses(cell.self_pos)
                normal_neighbors = self.get_neighbor_cells(normals)
                diag_neighbors = self.get_neighbor_cells(diags)
                cell.add_normal_neighbor(normal_neighbors)
                cell.add_diag_neighbor(diag_neighbors)
        
    #returns 2 dicts with them holding the diagonal and normal axis neighbors of the given index (pos) with their corresponding direction as the key value    
    def get_neighbor_poses(self, pos):
        adjacents = [[[0,0],[0,0],[0,0]],
                    [[0,0],[0,0],[0,0]],
                    [[0,0],[0,0],[0,0]]]
        vals = [-1,0,1]
        
        #this enters the adjacent coordinates
        for x, row in enumerate(adjacents):
            for y, col in enumerate(row):
                col[0] = pos[0] + vals[x]
                col[1] = pos[1] + vals[y]
        
        neighbors = {}
        for row in adjacents:
            for col in row:
                if not(col[0] < 0 or col[1] < 0 or col[0] >= self.board_dims[0] or col[1] >= self.board_dims[1] or col == pos):
                    if col[0] != pos[0] and col[1] != pos[1]:
                        if col[0] < pos[0] and col[1] < pos[1]:
                            neighbors.update({'top left':col})
                        elif col[0] > pos[0] and col[1] < pos[1]:
                            neighbors.update({'top right':col})
                        elif col[0] < pos[0] and col[1] > pos[1]:
                            neighbors.update({'bottom left':col})
                        elif col[0] > pos[0] and col[1] > pos[1]:
                            neighbors.update({'bottom right':col})
                    else:
                        if col[0] > pos[0]:
                            neighbors.update({'right':col})
                        elif col[1] > pos[1]:
                            neighbors.update({'top':col})
                        elif col[0] < pos[0]:
                            neighbors.update({'left':col})
                        elif col[1] < pos[1]:
                            neighbors.update({'bottom':col})
        
        #returns dict of diags and normals, using list comprehension
        normal_neighbors = ['right','left','top','bottom']
        diag_neighbors = ['top right','top left','bottom right', 'bottom left']
        #returns diag, normal
        return {location:neighbors[location] for location in neighbors if location in diag_neighbors}, {location:neighbors[location] for location in neighbors if location in normal_neighbors}

    #returns a list of cells objects from the given dict neighbors
    def get_neighbor_cells(self, neighbors):
        cells = {}
        for neighbor in neighbors:
            row = neighbors[neighbor][0]
            col = neighbors[neighbor][1]
            cells.update({neighbor:self.board[row][col]})
        return cells
    
    def flip_cell(self, idx, token, value):
        self.board[idx[0]][idx[1]].flip(value, token)
        
    def get_cell(self, idx):
        return self.board[idx[0]][idx[1]]
    
    def get_board_dims(self):
        return self.board_dims
    
    def print_board_neighbors(self):
        for row in self.board:
            for cell in row:
                normal_poses, diag_poses = cell.get_neighbor_poses()
                print(f'position : {cell.self_pos} ||| normal neighbors: {normal_poses} ||| diag neighbors: {diag_poses}')
                
    def print_board(self):
        count = 0
        frmt = "{:>3}"*len(self.board[0])
        print(f"  {frmt.format(*[str(x) for x in range(self.board_dims[0])])}")
        for row in self.board:
            val_list = [col.value for col in row]
            print(f"{count} {frmt.format(*val_list)}")
            # print()
            count+=1

In [12]:
class Othello():
    def __init__(self, board_dims):
        self.board = Board(board_dims=board_dims)
        self.players = {'player 1':{'token': 'X', 'value':1}, 'player 2':{'token':'O', 'value':-1}}
        # self.possible_moves_x = {} #key is the index where x can play, the value are the indices of the cells which have to be flipped when key is chosen, non inclusive of the key
        # self.possible_moves_O = {} # ''
        self.possible_moves = {'player 1': '', 'player 2': ''} #key: player, val is dict of moves and pieces that flip when taken
        self.taken_moves = [] #list of cells which have been taken
        # self.__setup__()
        self.__test_setup__()
        self.game_has_ended = False
        self.update_possible_moves()
        
    
    def __setup__(self):
        #O is white, X is black
        #starting board:
        # 0|1|2|3|4|5|6|7
        #0-|-|-|-|-|-|-|-
        #1-|-|-|-|-|-|-|-
        #2-|-|-|-|-|-|-|-
        #3-|-|-|O|X|-|-|-
        #4-|-|-|X|O|-|-|-
        #5-|-|-|-|-|-|-|-
        #6-|-|-|-|-|-|-|-
        #7-|-|-|-|-|-|-|-
        self.board.flip_cell(idx=[3,3], token=self.players['player 1']['token'], value = self.players['player 1']['value'])
        self.board.flip_cell(idx=[4,4], token=self.players['player 1']['token'], value = self.players['player 1']['value'])
        self.board.flip_cell(idx=[3,4], token=self.players['player 2']['token'], value = self.players['player 2']['value'])
        self.board.flip_cell(idx=[4,3], token=self.players['player 2']['token'], value = self.players['player 2']['value'])
        self.taken_moves.append(self.board.get_cell([3,3]))    
        self.taken_moves.append(self.board.get_cell([4,4]))    
        self.taken_moves.append(self.board.get_cell([4,3]))    
        self.taken_moves.append(self.board.get_cell([3,4]))    

        self.update_possible_moves()
    
    def __test_setup__(self):
        #     0  1  2  3  4  5  6  7
        # 0   -  -  -  -  -  -  -  -`
        # 1   -  -  -  -  -  -  -  -
        # 2   -  - -1 -1 -1  -  -  -
        # 3   -  - -1 -1  1  1  -  -
        # 4   -  -  1  1 -1  1  -  -
        # 5   -  -  -  1 -1 -1  -  -
        # 6   -  -  -  -  -  -  -  -
        # 7   -  -  -  -  -  -  -  -`
        
        player_1_t = self.players['player 1']['token']
        player_1_v = self.players['player 1']['value']
        player_2_t = self.players['player 2']['token']
        player_2_v = self.players['player 2']['value']
        
        self.board.flip_cell(idx=[2,2], token=player_2_t, value = player_2_v)
        self.board.flip_cell(idx=[2,3], token=player_2_t, value = player_2_v)
        self.board.flip_cell(idx=[2,4], token=player_2_t, value = player_2_v)
        self.board.flip_cell(idx=[3,2], token=player_2_t, value = player_2_v)
        self.board.flip_cell(idx=[3,3], token=player_2_t, value = player_2_v)
        self.board.flip_cell(idx=[4,4], token=player_2_t, value = player_2_v)
        self.board.flip_cell(idx=[5,4], token=player_2_t, value = player_2_v)
        self.board.flip_cell(idx=[5,5], token=player_2_t, value = player_2_v)
        self.board.flip_cell(idx=[4,2], token=player_1_t, value = player_1_v)
        self.board.flip_cell(idx=[4,3], token=player_1_t, value = player_1_v)
        self.board.flip_cell(idx=[5,3], token=player_1_t, value = player_1_v)
        self.board.flip_cell(idx=[4,5], token=player_1_t, value = player_1_v)
        self.board.flip_cell(idx=[3,5], token=player_1_t, value = player_1_v)
        self.board.flip_cell(idx=[3,4], token=player_1_t, value = player_1_v)
        self.taken_moves.append(self.board.get_cell([2,2]))    
        self.taken_moves.append(self.board.get_cell([2,3]))    
        self.taken_moves.append(self.board.get_cell([2,4]))    
        self.taken_moves.append(self.board.get_cell([3,2]))   
        self.taken_moves.append(self.board.get_cell([3,3]))    
        self.taken_moves.append(self.board.get_cell([4,4]))    
        self.taken_moves.append(self.board.get_cell([5,4]))    
        self.taken_moves.append(self.board.get_cell([5,5]))   
        self.taken_moves.append(self.board.get_cell([4,2]))    
        self.taken_moves.append(self.board.get_cell([4,3]))
        self.taken_moves.append(self.board.get_cell([5,3]))
        self.taken_moves.append(self.board.get_cell([4,5]))
        self.taken_moves.append(self.board.get_cell([3,5]))
        self.taken_moves.append(self.board.get_cell([3,4]))
    
    def update_possible_moves(self):
        for player in self.players.keys():
            self.possible_moves[player] = self.find_possible_moves(player)
            
    #takes string player parameter
    def find_possible_moves(self, player):
        opp_player_value = self.players['player 1']['value'] if player != 'player 1' else self.players['player 2']['value']
        player_val = self.players[player]['value']
        empty_adj_cells = self.get_empty_adj_cells(opp_player_value)
        
        #this dict will be filled with the empty cell : dict of direction and cells which are flipped in that direction
        #all values in the second dict will be the values which will need to be flipped if that empty cell move is chosen
        moves = {}
        for empty_cell in empty_adj_cells:
            moves.update({empty_cell:self.find_cells_flipped(player_val, empty_cell, opp_player_value)})
        
        out = {}
        for key in moves.keys():
            if moves[key]:
                # print(moves[key])
                out.update({key:moves[key]})
        # print(out)
        return out
    
    def get_empty_adj_cells(self, opp_player_value):
        all_empty_adj_cells = []
        
        for move in self.taken_moves:
            if move.value == opp_player_value:
                all_empty_adj_cells.append(move.get_empty_neighbors())
                
        flat = [empty for empty_adj in all_empty_adj_cells for empty in empty_adj]
        out = []
        [out.append(x) for x in flat if x not in out]
        return out
        
    def find_cells_flipped(self, player_val, cur_cell, opp_player_value):
        directions = ['top','bottom','right','left','top right','bottom left','bottom right','top left']
        flipped_cells = []
        for direction in directions:
            ret_val = self.check_dir(player_val, cur_cell, opp_player_value, direction)
            if ret_val:
                flipped_cells = flipped_cells + ret_val
        return flipped_cells
    
    def check_dir(self, player_val, cur_cell, opp_player_value, direction):
        next_cell = cur_cell.get_neighbor_cell_in_direction(direction)
        path = []
        if isinstance(next_cell, bool) or next_cell == False:
            return []
        
        # print(f'check_dir start cell pos: {cur_cell.self_pos} | start cell val: {cur_cell.value} | next cell: {next_cell.value} | next call loc: {next_cell.self_pos} | opp_player_value: {opp_player_value} | player value: {player_val}')
        while next_cell.value == opp_player_value:
            # print(f'appended {next_cell.value} {next_cell.self_pos}')
            path.append(next_cell)
            next_cell = next_cell.get_neighbor_cell_in_direction(direction)
            if isinstance(next_cell, bool) or next_cell == False:
                return []
        # if next_cell.value != '-':
        #     print(f'out of while next_cell: {next_cell.value} | player val: {player_val} | opp: {opp_player_value}')
            
        if next_cell.value == player_val:
            # print(f'returning: {path}')
            return path
        else:
            return []
        
    def flip_cells(self, move_taken, player):
        
        player_val = self.players[player]['value']
        player_token = self.players[player]['token']
        
        # self.print_possible_moves(player)
        # print(self.possible_moves[player])
        # print(move_taken)
        flip_idx_list = self.possible_moves[player][self.board.get_cell(move_taken)]
        # print(flip_idx_list)
        self.board.get_cell(move_taken).flip(player_val, player_token)
        self.taken_moves.append(self.board.get_cell(move_taken))
        for cell in flip_idx_list:
            self.board.get_cell(cell.self_pos).flip(player_val, player_token)
        
    def take_turn(self, num, player):
        idx = list(self.possible_moves[player].keys())[int(num)-1]
        print(num)
        print(idx)
        self.flip_cells(idx.self_pos, player)
        self.update_possible_moves()
        opp_player = list(self.players.keys())
        opp_player.remove(player)
        opp_player = opp_player[0]
        self.check_end(opp_player)
        #at end of check true or false, and then append if true else dont
        
    
    def print_possible_moves(self, player):
        moves = self.possible_moves[player]
        count = 1
        for move in moves:
            print(f'move {count} | cell idx: {move.self_pos} | flipped cells: {moves[move]}')
            count += 1
        
    def display(self, player):
        self.board.print_board()
        self.print_possible_moves(player)
        
    def check_end(self, player):
        players = list(self.players.keys())
        players.remove(player)
        max_count = self.board.get_board_dims()[0] * self.board.get_board_dims()[1]
        print(f'max count: {max_count} | len pos moves: {len(self.possible_moves[player])} | len taken moves: {len(self.taken_moves)} | player: {player}')
        if len(self.possible_moves[player]) == 0 or len(self.taken_moves) == max_count:
            self.display(players[0])
            self.game_end(players[0])
            
    def game_end(self, player):
        self.game_has_ended = True
        print(f'game had ended, player: {player} has won')

In [13]:
new_board = Othello([8,8])
players = ['player 1', 'player 2']
x = 0
while new_board.game_has_ended == False:
    print(f"player {players[x%2]}'s turn")
    new_board.display(players[x%2])
    uin = input('Please choose a move from the possible moves, please enter position/idx')
    # uin = [int(uin[0]), int(uin[2])]
    new_board.take_turn(uin, players[x%2])
    x+=1
# new_board.display('player 1')
# new_board.take_turn([3,5], 'player 1')
# new_board.display('player 2')

player player 1's turn
    0  1  2  3  4  5  6  7
0   -  -  -  -  -  -  -  -
1   -  -  -  -  -  -  -  -
2   -  - -1 -1 -1  -  -  -
3   -  - -1 -1  1  1  -  -
4   -  -  1  1 -1  1  -  -
5   -  -  -  1 -1 -1  -  -
6   -  -  -  -  -  -  -  -
7   -  -  -  -  -  -  -  -
move 1 | cell idx: [2, 1] | flipped cells: [<__main__.Cell object at 0x0000014644F41910>]
move 2 | cell idx: [1, 2] | flipped cells: [<__main__.Cell object at 0x0000014644F42FD0>, <__main__.Cell object at 0x0000014644F41910>, <__main__.Cell object at 0x0000014644F43D50>]
move 3 | cell idx: [3, 1] | flipped cells: [<__main__.Cell object at 0x0000014644F41910>, <__main__.Cell object at 0x0000014644F40950>]
move 4 | cell idx: [1, 3] | flipped cells: [<__main__.Cell object at 0x0000014644F43D50>, <__main__.Cell object at 0x0000014644F40950>, <__main__.Cell object at 0x0000014644F40E50>]
move 5 | cell idx: [1, 4] | flipped cells: [<__main__.Cell object at 0x0000014644F40E50>]
move 6 | cell idx: [1, 5] | flipped cells: [<__main__.

In [291]:
h = {'k':{'x':1}, 'b':{}}
print(f'k: {len(h["k"])} | b:{len(h["b"])}')

k: 1 | b:0


In [93]:
h = {'d':[0,1],'h':[1,5],'x':[5,10]}
k = ['d','h']
for value, key in enumerate(h):
    print(f"key: {key} | value: {value}")

key: d | value: 0
key: h | value: 1
key: x | value: 2


In [65]:
board_dims = [8,8]
cell_board = [[Cell(pos=[x,y],board_dims=board_dims) for x in range(board_dims[0])] for y in range(board_dims[1])]

In [70]:
def get_neighbor_poses(pos, board_dims):
    adjacents = [[[0,0],[0,0],[0,0]],
                 [[0,0],[0,0],[0,0]],
                 [[0,0],[0,0],[0,0]]]
    vals = [-1,0,1]
    
    #this enters the adjacent coordinates
    for x, row in enumerate(adjacents):
        for y, col in enumerate(row):
            col[0] = pos[0] + vals[x]
            col[1] = pos[1] + vals[y]
    
    neighbors = {}
    for row in adjacents:
        for col in row:
            if not(col[0] < 0 or col[1] < 0 or col[0] >= board_dims[0] or col[1] >= board_dims[1] or col == pos):
                if col[0] != pos[0] and col[1] != pos[1]:
                    # print(f'col:{col} | pos:{pos}')
                    # print(col[0] != pos[0] and col[1] != pos[1])
                    # diag_neighbors.append(col)
                    if col[0] < pos[0] and col[1] < pos[1]:
                        neighbors.update({'top left':col})
                    elif col[0] > pos[0] and col[1] < pos[1]:
                        neighbors.update({'top right':col})
                    elif col[0] < pos[0] and col[1] > pos[1]:
                        neighbors.update({'bottom left':col})
                    elif col[0] > pos[0] and col[1] > pos[1]:
                        neighbors.update({'bottom right':col})
                else:
                    if col[0] > pos[0]:
                        neighbors.update({'right':col})
                    elif col[1] > pos[1]:
                        neighbors.update({'top':col})
                    elif col[0] < pos[0]:
                        neighbors.update({'left':col})
                    elif col[1] < pos[1]:
                        neighbors.update({'bottom':col})
    
    #returns dict of diags and normals, using list comprehension
    normal_neighbors = ['right','left','top','bottom']
    diag_neighbors = ['top right','top left','bottom right', 'bottom left']
    #returns diag, normal
    return {location:neighbors[location] for location in neighbors if location in diag_neighbors}, {location:neighbors[location] for location in neighbors if location in normal_neighbors}

def get_neighbor_cells(neighbors, cell_board):
    cells = []
    for neighbor in neighbors:
        # print(neighbor)
        row = neighbors[neighbor][0]
        col = neighbors[neighbor][1]
        cells.append(cell_board[row][col])
    return cells



In [67]:
#build board with all dependencies, like nieghbors and locations
# normal_n = ['right', 'top','left','bottom']
for row in cell_board:
    for cell in row:
        diags, normals = get_neighbor_poses(cell.self_pos, board_dims=board_dims)
        print(f'normal: {normals} | diag: {diags}')
        normal_neighbors = get_neighbor_cells(normals, cell_board=cell_board)
        diag_neighbors = get_neighbor_cells(diags, cell_board=cell_board)
        print(f'normal cells: {[c.self_pos for c in normal_neighbors]} | diag cells: {[c.self_pos for c in diag_neighbors]}')
        # break
        cell.add_normal_neighbor(normal_neighbors)
        cell.add_diag_neighbor(diag_neighbors)

normal: {'top': [0, 1], 'right': [1, 0]} | diag: {'bottom right': [1, 1]}
normal cells: [[1, 0], [0, 1]] | diag cells: [[1, 1]]
normal: {'left': [0, 0], 'top': [1, 1], 'right': [2, 0]} | diag: {'bottom left': [0, 1], 'bottom right': [2, 1]}
normal cells: [[0, 0], [1, 1], [0, 2]] | diag cells: [[1, 0], [1, 2]]
normal: {'left': [1, 0], 'top': [2, 1], 'right': [3, 0]} | diag: {'bottom left': [1, 1], 'bottom right': [3, 1]}
normal cells: [[0, 1], [1, 2], [0, 3]] | diag cells: [[1, 1], [1, 3]]
normal: {'left': [2, 0], 'top': [3, 1], 'right': [4, 0]} | diag: {'bottom left': [2, 1], 'bottom right': [4, 1]}
normal cells: [[0, 2], [1, 3], [0, 4]] | diag cells: [[1, 2], [1, 4]]
normal: {'left': [3, 0], 'top': [4, 1], 'right': [5, 0]} | diag: {'bottom left': [3, 1], 'bottom right': [5, 1]}
normal cells: [[0, 3], [1, 4], [0, 5]] | diag cells: [[1, 3], [1, 5]]
normal: {'left': [4, 0], 'top': [5, 1], 'right': [6, 0]} | diag: {'bottom left': [4, 1], 'bottom right': [6, 1]}
normal cells: [[0, 4], [1, 

In [68]:
for row in cell_board:
    for cell in row:
        normal_poses, diag_poses = cell.get_neighbor_poses()
        print(f'position : {cell.self_pos} ||| normal neighbors: {normal_poses} ||| diag neighbors: {diag_poses}')

position : [0, 0] ||| normal neighbors: [[1, 0], [0, 1]] ||| diag neighbors: [[1, 1]]
position : [1, 0] ||| normal neighbors: [[0, 0], [1, 1], [0, 2]] ||| diag neighbors: [[1, 0], [1, 2]]
position : [2, 0] ||| normal neighbors: [[0, 1], [1, 2], [0, 3]] ||| diag neighbors: [[1, 1], [1, 3]]
position : [3, 0] ||| normal neighbors: [[0, 2], [1, 3], [0, 4]] ||| diag neighbors: [[1, 2], [1, 4]]
position : [4, 0] ||| normal neighbors: [[0, 3], [1, 4], [0, 5]] ||| diag neighbors: [[1, 3], [1, 5]]
position : [5, 0] ||| normal neighbors: [[0, 4], [1, 5], [0, 6]] ||| diag neighbors: [[1, 4], [1, 6]]
position : [6, 0] ||| normal neighbors: [[0, 5], [1, 6], [0, 7]] ||| diag neighbors: [[1, 5], [1, 7]]
position : [7, 0] ||| normal neighbors: [[0, 6], [1, 7]] ||| diag neighbors: [[1, 6]]
position : [0, 1] ||| normal neighbors: [[0, 0], [2, 0], [1, 1]] ||| diag neighbors: [[0, 1], [2, 1]]
position : [1, 1] ||| normal neighbors: [[1, 0], [0, 1], [2, 1], [1, 2]] ||| diag neighbors: [[0, 0], [2, 0], [0, 

In [ ]:
def find_flank(cell_board, pos, player):
    

In [1]:
class Othello_old():
    def __init__(self):
        self.board = [['-' for _ in range(8)] for _ in range(8)]
        self.players = {'p1':'O', 'p2':'X'}
        #o is white, x is black
        #starting board:
        # 0|1|2|3|4|5|6|7
        #0-|-|-|-|-|-|-|-
        #1-|-|-|-|-|-|-|-
        #2-|-|-|-|-|-|-|-
        #3-|-|-|O|X|-|-|-
        #4-|-|-|X|O|-|-|-
        #5-|-|-|-|-|-|-|-
        #6-|-|-|-|-|-|-|-
        #7-|-|-|-|-|-|-|-
        self.board[3][3] = self.players['p1']
        self.board[4][4] = self.players['p1']
        self.board[3][4] = self.players['p2']
        self.board[4][3] = self.players['p2']
        
        self.moves_made = []
        self.open_vals = [[str(row)+str(col) for col in range(len(self.board))] for row in range(len(self.board))]
        #list of start_start_start_start_start_poses that are on the edges of the pieces placed, a Set of all valid spaces
        #updates each time a piece is placed
        self.edge_spaces = []
        
        self.score = {list(self.players.keys())[0]:0, list(self.players.keys())[1]:0}
        
        self.terminated = False
        #struct adj_to, poses
        #{x:{adj_to:[adjacent open poses], ...}, o:{adj_to:[adjacent open poses]}}
        self.adjacent_positions = {self.players['p1']:{[3,3]:[[3,2],[2,3],[2,2]], [4,4]:[[4,5],[5,4],[5,5]]}, self.players['p2']:{[3,4]:[[4,2],[5,3],[5,2]],[4,3]:[[2,4],[3,5],[2,5]]}}
    
    def __print_board__(self):
        for row in self.board:
            print(row)
    
    #start_pos is a list [row, col], player is 'p1' or 'p2'
    def take_turn(self, pos, player):
        if self.check_valid(self.board, pos):
            self.place_piece(pos, player)
            self.turn_pieces(pos)
            if self.check_end(player):
                self.end()
    
    #have list of valid positions for each player
    #then have list of vals that will be flipped for each valid position
    #update both lists each time player makes move
    #use the list to see if valid input and if the player can make move
    
    def get_valid_positions(self, player):
        player_list = list(self.player.keys())
        player_list.remove(player)
        opposing_player = player_list[0]
        opposing_player_mark = self.players[opposing_player]
        
        valid_positions = []
        
        for key, value in self.adjacent_positions[opposing_player].items():
            for val in value:
                if self.is_flank(val, key)
        
        return #list of valid positions ie pos (x,y)
    
    #returns bool and a list of values changed if it is a flank
    def is_flank(self, pos, adj_to):
        case
            
    def find_diag_dir(pos, adj_to):
        if pos[0] > adj_to[0]:
            if pos[1] > adj_to[1]:
                return 'bottom right'
            elif pos[1] < adj_to[1]:
                return 'top right'
        elif pos[0] < adj_to[0]:
            if pos[1] > adj_to[1]:
                return 'bottom left'
            elif pos[1] < adj_to[1]:
                return 'top left'
        else:
            return 'not diag'
        
    
    def check_diag(self, start_pos, adj_pos, player, opposing_player):
        top_lmin_val = start_pos[0] if start_pos[0] <= start_pos[1] else start_pos[1]
        top_rmin_val = start_pos[0] if start_pos[0] <= len(self.board)-start_pos[1] else start_pos[1]
        bottom_lmin_val = start_pos[0] if len(self.board)-start_pos[0] <= start_pos[1] else start_pos[1]
        bottom_rmin_val = start_pos[0] if len(self.board)-start_pos[0] <= len(self.board) - start_pos[1] else start_pos[1]
        
        val_list = [[top_lmin_val,0,-1], [top_rmin_val,0,-1], [bottom_lmin_val,len(self.board),1], [bottom_rmin_val,len(self.board),1]]
        
        
        #top left diag
        if start_pos[0] > adj_pos[0] and start_pos[1] > adj_pos[1]:
            for xy in range(val_list[0][0],val_list[0][1],val_list[0][2]):
                if self.board[xy][xy] == self.players[player]:
                    break
                elif self.board[xy][xy] == self.players[opposing_player]:
                    change_able_poses.append([xy,xy])
                else:
                    for pop in range(pop_counter):
                        change_able_poses.pop()
                        break
    
    def get_valid_adjacents(self, pos, opposing_player_mark):
        adjacents = [[[0,0],[0,0],[0,0]],
                     [[0,0],[0,0],[0,0]],
                     [[0,0],[0,0],[0,0]]]
        vals = [-1,0,1]
        
        #this enters the adjacent coordinates
        for x, row in enumerate(adjacents):
            for y, col in enumerate(row):
                col[0] = pos[0] + vals[x]
                col[1] = pos[1] + vals[y]
        
        for row in adjacents:
            for val in row:
                if self.within_range(val):
                    if self.board[val[0]][val[1]] == opposing_player_mark:
                        val[0] = -10
                        val[1] = -10
                else:
                    val[0] = -10
                    val[1] = -10
                    
                        
        return adjacents
    
    def within_range(self, pos):
        if (pos[0] >= 0 and pos[0] <= len(self.board)) and (pos[1] >= 0 and pos[1] <= len(self.board)):
            return True
        else:
            return False
    
    def check_valid(self, pos, player):
        if self.board[pos[0]][pos[1]] == '-':
            if self.check_adjacent(pos, player):
                return True
        else:
            False
    
    def check_adjacent(self, pos, player):
        adjacents = [[[0,0],[0,0],[0,0]],
                     [[0,0],[0,0],[0,0]],
                     [[0,0],[0,0],[0,0]]]
        vals = [-1,0,1]
        adjacent_exits = False
        #this enters the adjacent coordinates
        for x, row in enumerate(adjacents):
            for y, col in enumerate(row):
                col[0] = pos[0] + vals[x]
                col[1] = pos[1] + vals[y]
        #this checks if the coordinates are valid, ie in the board, and if there exists any adjacent values, since if there are none, the piece cannot be played there
        for row in adjacents:
            for col in row:
                if col[0] < 0 or col[1] < 0:
                    continue
                else:
                    if self.board[col[0]][col[1]] != self.players[player] or self.board[col[0]][col[1]] == '-':
                        adjacent_exits = True
                    else:
                        continue
                    
        return adjacent_exits
    
    def place_piece(self, pos, player):
        self.board[pos[0]][pos[1]] == self.board[player]
        self.update_mtaken_and_open(pos)
        return
    
    def update_mtaken_and_open(self, pos):
        self.moves_made.append(pos)
        val = str(pos[0]) + str(pos[1])
        for row in self.open_vals:
            if val in row:
                row.remove(val)    
                
    def turn_pieces(self, pos, player):
        opposing_player = 'p1' if player != 'p1' else 'p2'
        #go left
        change_able_poses = []
        pop_counter = 0
        for x in range(pos[0], 0 , -1):
            if self.board[x][pos[1]] == self.players[player]:
                break
            elif self.board[x][pos[1]] == self.players[opposing_player]:
                pop_counter += 1
                change_able_poses.append([x,pos[1]])
            else:
                for pop in range(pop_counter):
                    change_able_poses.pop()
                break
        #go right
        for x in range(pos[0], len(self.board)):
            if self.board[x][pos[1]] == self.players[player]:
                break
            elif self.board[x][pos[1]] == self.players[opposing_player]:
                change_able_poses.append([x,pos[1]])
            else:
                for pop in range(pop_counter):
                    change_able_poses.pop()
                break
        #go up
        for y in range(pos[1], 0 , -1):
            if self.board[pos[0]][y] == self.players[player]:
                break
            elif self.board[pos[0]][y] == self.players[opposing_player]:
                change_able_poses.append([pos[0],y])
            else:
                for pop in range(pop_counter):
                    change_able_poses.pop()
                break
        #go down
        for y in range(pos[1], len(self.board)):
            if self.board[pos[0]][y] == self.players[player]:
                break
            elif self.board[pos[0]][y] == self.players[opposing_player]:
                change_able_poses.append([pos[0],y])
            else:
                for pop in range(pop_counter):
                    change_able_poses.pop()
                break
            
        #go diagonals
        top_lmin_val = pos[0] if pos[0] <= pos[1] else pos[1]
        top_rmin_val = pos[0] if pos[0] <= len(self.board)-pos[1] else pos[1]
        bottom_lmin_val = pos[0] if len(self.board)-pos[0] <= pos[1] else pos[1]
        bottom_rmin_val = pos[0] if len(self.board)-pos[0] <= len(self.board) - pos[1] else pos[1]
        
        val_list = [[top_lmin_val,0,-1], [top_rmin_val,0,-1], [bottom_lmin_val,len(self.board),1], [bottom_rmin_val,len(self.board),1]]
        
        for val in val_list:
            for xy in range(val[0],val[1],val[2]):
                if self.board[xy][xy] == self.players[player]:
                    break
                elif self.board[xy][xy] == self.players[opposing_player]:
                    change_able_poses.append([xy,xy])
                else:
                    for pop in range(pop_counter):
                        change_able_poses.pop()
                        break
                    
        for change in change_able_poses:
            self.board[change[0]][change[1]] = self.players[player]
            self.score[player] += 1
        
        # for xy in range(top_lmin_val ,0,-1):
        #     if self.board[xy][xy] == player[player]:
        #         break
        #     elif self.board[xy][xy] == player[opposing_player]:
        #         change_able_poses.append([xy,xy])
        #     else:
        #         for pop in range(pop_counter):
        #             change_able_poses.pop()
        #             break
        # #bottom left
        # for xy in range(bottom_lmin_val, 0, -1):
        #     if self.board[xy][xy] == player[player]:
        #         break
        #     elif self.board[xy][xy] == player[opposing_player]:
        #         change_able_poses.append([xy,xy])
        #     else:
        #         for pop in range(pop_counter):
        #             change_able_poses.pop()
        #             break
        # #top right
        # for xy in range(top_rmin_val, len(self.board)):
        #     if self.board[xy][xy] == player[player]:
        #         break
        #     elif self.board[xy][xy] == player[opposing_player]:
        #         change_able_poses.append([xy,xy])
        #     else:
        #         for pop in range(pop_counter):
        #             change_able_poses.pop()
        #             break
        # #bottom right
        # for xy in range(bottom_rmin_val, len(self.board)):
        #     if self.board[xy][xy] == player[player]:
        #         break
        #     elif self.board[xy][xy] == player[opposing_player]:
        #         change_able_poses.append([xy,xy])
        #     else:
        #         for pop in range(pop_counter):
        #             change_able_poses.pop()
        #             break
    
    def check_end(self, player):
        neg_player_list = list(self.players.keys())
        neg_player_list = neg_player_list.remove(player)
        next_player = neg_player_list[0]
        if len(self.open_vals) == 0:
            return True
        elif any([self.check_valid([int(pos[0]),int(pos[1])], next_player) for pos in self.open_vals]) == True:
            return False
        else:
            return True
            
    def end(self):
        if self.score['p1'] > self.score['p2']:
            winner = 'p1'
        elif self.score['p1'] < self.score['p2']:
            winner = 'p2'
        else:
            winner = 'tie'
        print(f'Game Over, Winner is {winner}')
        self.terminated = True

In [7]:
h = [['-' for _ in range(8)] for _ in range(8)]
h[0][1]= 'O'
print(h)

[['-', 'O', '-', '-', '-', '-', '-', '-'], ['-', '-', '-', '-', '-', '-', '-', '-'], ['-', '-', '-', '-', '-', '-', '-', '-'], ['-', '-', '-', '-', '-', '-', '-', '-'], ['-', '-', '-', '-', '-', '-', '-', '-'], ['-', '-', '-', '-', '-', '-', '-', '-'], ['-', '-', '-', '-', '-', '-', '-', '-'], ['-', '-', '-', '-', '-', '-', '-', '-']]


In [4]:
va = [[str(row)+str(col) for col in range(8)] for row in range(8)]
print(va)


[['00', '01', '02', '03', '04', '05', '06', '07'], ['10', '11', '12', '13', '14', '15', '16', '17'], ['20', '21', '22', '23', '24', '25', '26', '27'], ['30', '31', '32', '33', '34', '35', '36', '37'], ['40', '41', '42', '43', '44', '45', '46', '47'], ['50', '51', '52', '53', '54', '55', '56', '57'], ['60', '61', '62', '63', '64', '65', '66', '67'], ['70', '71', '72', '73', '74', '75', '76', '77']]


In [6]:
for row in va:
    if '00' in row:
        row.remove('00')

In [18]:
def get_input(open_places):
    print('open spaces are:')
    for row in open_places:
        print(row)
    uinput = input("where do you want to place your piece")
    conv_uinput = str(uinput[0]) + str(uinput[1])
    
    if conv_uinput not in open_places:
        while conv_uinput not in open_places:
            uinput = input('not valid, choose a valid space')
            conv_uinput = str(uinput[0]) + str(uinput[1])
    
    return uinput

In [19]:
game = Othello()
players = ['p1','p2']

turn_counter = 0
while game.terminated == False:
    uinput = get_input(game.open_vals)
    game.turn_pieces(uinput, players[turn_counter%2])
    game.__print_board__()
    turn_counter += 1

open spaces are:
['00', '01', '02', '03', '04', '05', '06', '07']
['10', '11', '12', '13', '14', '15', '16', '17']
['20', '21', '22', '23', '24', '25', '26', '27']
['30', '31', '32', '33', '34', '35', '36', '37']
['40', '41', '42', '43', '44', '45', '46', '47']
['50', '51', '52', '53', '54', '55', '56', '57']
['60', '61', '62', '63', '64', '65', '66', '67']
['70', '71', '72', '73', '74', '75', '76', '77']


IndexError: string index out of range